<a href="https://colab.research.google.com/github/Riponcs/Cn2Estimation/blob/main/Cn2_Estimation_from_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# $C_n^2$ Estimation From Image Sequences
Code Author: Ripon Kumar Saha \
Imaging Lyceum Lab (ASU) \
[Paper: Turbulence strength $C_n^2$
 estimation from video using physics-based deep learning](https://opg.optica.org/oe/fulltext.cfm?uri=oe-30-22-40854&id=511116)

## Download Dataset: [DropBox Download](https://www.dropbox.com/s/f8uqekwxy2qotfb/Turbulence_Dataset.zip)



In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import pandas as pd
import numpy as np
from PIL import Image
import os
import glob

from skimage import io, transform
import seaborn as sns
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import ipywidgets as widgets

from datetime import date, datetime
import math, time
from tqdm import tqdm

In [ ]:
# Download the Dataset form internet (if not already downloaded)
if not os.path.exists('Turbulence_Dataset'):
  !wget -O TestRipon.zip "https://www.dropbox.com/s/f8uqekwxy2qotfb/Turbulence_Dataset.zip?dl=1"
  !unzip TestRipon.zip

%%hide output

In [ ]:
DatasetName = widgets.Dropdown(options=['July', 'October', 'Both'], value='October', description='Number:', disabled=False)
display(DatasetName)

Dropdown(description='Number:', index=1, options=('July', 'October', 'Both'), value='October')

In [ ]:
JulyDataset, OctoberDataset, All = 0, 0, 0

print('We are using Dataset: ', DatasetName.value)

if DatasetName.value == 'July':
    JulyDataset = 1
if DatasetName.value == 'October':
    OctoberDataset = 1
if DatasetName.value == 'Both':
    All = 1

In [ ]:
if JulyDataset==1:
#     NikPath = "H:/HDD1/Field Test 7-29-21/Nikon Photos/ManualAnnotated/Patch Size/Size_ 256_256/**/*.JPG"
    NikPath = "Turbulence_Dataset/Image_Dataset/July_Crop_Stabilized/**/*.JPG"
    cn2path = ["Turbulence_Dataset/LogFiles/July/20210729_180021.log"] 
    
    
    cndfList = [pd.read_csv(a, sep=r'\t', engine='python', encoding= 'unicode_escape') for a in cn2path]
    cndf = pd.concat([a.iloc[1:] for a in cndfList])
    cndf = cndf.iloc[128:296]#.iloc[::2]
    cndf.drop(cndf.index[cndf['Cn2'] == '0.000000'], inplace=True)

if OctoberDataset==1:
#     NikPath = "H:/HDD1/Crop/Field Test 10-28-21-object-multi-MotionW-WO/WithPlatformCorrected/Nikon Photos/**/*.JPG"
    NikPath = "Turbulence_Dataset/Image_Dataset/October_Corp_Stabilized/**/*.JPG"
    cn2path = ["Turbulence_Dataset/LogFiles/October/20211028_180021.log","Turbulence_Dataset/LogFiles/October/20211029_180021.log"]
    distance = 1450 

    
    cndfList = [pd.read_csv(a, sep=r'\t', engine='python', encoding= 'unicode_escape') for a in cn2path]#[1]
    cndf = pd.concat([a.iloc[1:] for a in cndfList])
    cndf = cndf.iloc[1206:].iloc[::2]
    
    
if All==1:
    NikPath = ["Turbulence_Dataset\Image_Dataset\July_Crop_Stabilized/**/*.JPG", "Turbulence_Dataset/Image_Dataset/October_Corp_Stabilized/**/*.JPG"]
    cn2path = ["Turbulence_Dataset/LogFiles/October/20211028_180021.log","Turbulence_Dataset/LogFiles/October/20211029_180021.log"]
    
    cndfList = [pd.read_csv(a, sep=r'\t', engine='python', encoding= 'unicode_escape') for a in cn2path]#[1]
    cndf = pd.concat([a.iloc[1:] for a in cndfList])
    cndfOct = cndf.iloc[1206:].iloc[::2]
    
    cn2path = ["H:/HDD1/Field Test 7-29-21/Scint Logs/20210729_180021.log"] 
    cndfList = [pd.read_csv(a, sep=r'\t', engine='python', encoding= 'unicode_escape') for a in cn2path]
    cndf = pd.concat([a.iloc[1:] for a in cndfList])
    cndf = cndf.iloc[128:296]#.iloc[::2]
    cndf.drop(cndf.index[cndf['Cn2'] == '0.000000'], inplace=True)
    cndf = pd.concat([cndf, cndfOct], ignore_index=True, sort=False)

## Combining all Images and Scintillometer Data Together in CSV file

In [ ]:
# Correcting the Date-Time on Scintillometer Data
for i in range(len(cndf)):
    sciPreviusTimstamp = datetime.strptime(cndf['DateTime'].iloc[i], "%Y/%m/%d %H:%M:%S").timestamp()-7*3600  # The existed Data was 7 Hours added
    cndf['DateTime'].iloc[i] = datetime.fromtimestamp(sciPreviusTimstamp)                                     # So, We don't really need to convert to timezone
    
Cn2Max, Cn2Min = cndf['Cn2'].astype(float).max(),  cndf['Cn2'].astype(float).min()
def Cn2deNorm(cn2):
    return cn2*(Cn2Max-Cn2Min)+Cn2Min

cndf['Cn2_Normalized'] = cndf['Cn2'].apply(lambda t: (float(t)-Cn2Min)/(Cn2Max - Cn2Min))

NumOfCn2 = len(cndf)
display(cndf)

In [ ]:
NikDf = pd.DataFrame({'Nikon Time': [], 'Nikon Path' : [], 'Nikon TimeStamp': []})

if All==1:
    imageList = glob.glob(NikPath[0]) + glob.glob(NikPath[1])
else:
    imageList = glob.glob(NikPath)
    
for i, name in enumerate(tqdm(imageList)):
    tStamp = Image.open(name)._getexif()[36867]
    thatTime = datetime.strptime(tStamp, '%Y:%m:%d %H:%M:%S')

    NikDf1 = pd.DataFrame({'Nikon Time': [thatTime], 'Nikon Path' : [name], 'Nikon TimeStamp': [tStamp]})
    NikDf = NikDf.append(NikDf1, ignore_index = True)
    
# Replace All Second values to Zeros, And based on that Change the TimeStamp
NikDf['Nikon Time'] = pd.to_datetime(NikDf['Nikon Time'])
NikDf['Nikon Time'] = NikDf['Nikon Time'].apply(lambda t: t.replace(second=0))
NikDf['Nikon TimeStamp'] = NikDf['Nikon Time'].apply(lambda t: t.timestamp())
NikDf['Nikon Crop'] = NikDf['Nikon Path'].apply(lambda t: t.replace("\\", '/'))
display(NikDf)

# Join Both Scintillometer and Nikon Image path into Same CSV File
NikSinDF = NikDf.join(cndf.set_index('DateTime'), on='Nikon Time', how='inner')
NikSinDF.reset_index(drop=True, inplace=True)
NikSinDF.to_csv(f'{DatasetName.value}_Combined.csv')
NikSinDF.sort_values(by="Nikon TimeStamp", axis=0, ascending=True, inplace=True)
display(NikSinDF)

October Dataset Start from index **26220**

In [ ]:
plt.rcParams['figure.figsize'] = (20,12)
# plt.yscale('log',base=10), plt.grid(alpha=0.1)
plt.plot(np.array(NikSinDF['Cn2'][:26219].astype(float))), plt.title('Cn2 Graph in Log Scale', size=20);

# Dataset Class

In [ ]:
from torch.utils.data import Dataset, DataLoader

class cn2Dataset(Dataset):
    
    def __init__(self, dataframe, transform=None):
        self.cubeCh = cubeChNum # 3
        self.features = dataframe[:self.cubeCh*math.floor(len(dataframe)/self.cubeCh)]
        self.root_dir = dataframe['Nikon Crop']
        self.transform = transform
        
        length = int(len(self.features)/self.cubeCh)
        self.data = np.zeros((length, self.cubeCh, 256, 256))

    def __len__(self):
        return int(len(self.features)/self.cubeCh)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        if self.data[idx,:,:].sum()==0:        
            Image20 = np.zeros((self.cubeCh,256,256))
            
            for j in range(self.cubeCh):
                img_name = self.root_dir[idx*self.cubeCh+j]
                image = io.imread(img_name)[:,:,0]
                Image20[j, :, :] = image

            assert len(Image20.shape) == 3 and Image20.shape[0] == self.cubeCh
            Image20 = Image20[:, :, :]/255.0
            self.data[idx,:, :,:] = Image20
        
        Image20 = self.data[idx,:, :,:]
        Image20 = torch.from_numpy(Image20)
        Image20 = Image20.float().to(device)

        cn2_features = self.features['Cn2_Normalized'][idx*self.cubeCh]#.iloc[idx*self.cubeCh, 2]
        cn2_features = torch.from_numpy(np.array(cn2_features, dtype=np.float32)).to(device)

        if self.transform:
            Image20 = self.transform(Image20)
            
        sample = {'image': Image20, 'features': cn2_features}
        return sample

# Some Parameters To Choose

In [ ]:
lrate = 1e-4            #1e-4 best for 10x10 kernel
epochCount = 1          # Idealy 30, For simplicity running for 1 epoch
batch_s = 1
cubeChNum = 3           #Number of Images
kernelSize = 10
AdditionalLayer = 0
GradientCh = 1
GraphPath = "Graph/"    # Path to save Graphs, Folder will be automatically created
TestPortion = 2
Subsample = 1           # Select every nth Images
Opposit = 0             # 0 -> Train on July and test on october, 1-> Opposit
distanceFactor = 0.5    # In transfer model, 2 -> Train on July and Test on October, 0.5 -> viseversa 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
os.makedirs(GraphPath, exist_ok=True)

# Dataloader

In [ ]:
# Add Training IDs and Test Ids Based on 180 Values
if DatasetName.value != 'Both':
    TotalDataset = cn2Dataset(dataframe = NikSinDF[::Subsample], transform=None)

    L = len(TotalDataset)
    train_ids = []
    valid_ids = []
    test_ids = []

    it = 0
    for a in range(L):
        if int(a*NumOfCn2/L) == it:  #NumOfCn2 = Number of Scintillometer Unique Values
            valid_ids.append(a)
            it = it+1
        elif a%TestPortion==0:  #How many portion to be used for Testing
            test_ids.append(a)
        else:
            train_ids.append(a)

    train_ids, valid_ids, test_ids = np.array(train_ids), np.array(valid_ids), np.array(test_ids)

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(TotalDataset, shuffle=False, batch_size=batch_s, sampler=train_subsampler)
    validloader = torch.utils.data.DataLoader(TotalDataset,shuffle=False, batch_size=batch_s, sampler=valid_subsampler)
    testloader = torch.utils.data.DataLoader(TotalDataset,shuffle=False, batch_size=batch_s, sampler=test_subsampler)

    print(f'Number of training Data = {len(train_ids)}, and Number of Test dataset = {len(test_ids)}')

# Extrapolation Data Loader

In [ ]:
# Add julying IDs and Test Ids Based on 180 Values
if DatasetName.value == 'Both':
    TotalDataset = cn2Dataset(dataframe = NikSinDF[::Subsample], transform=None)

    L = len(TotalDataset)
    july_ids = []
    october_ids = []

    it = 0
    for a in range(L):
        if cubeChNum*a<26220:  #NumOfCn2 = Number of Scintillometer Unique Values
            july_ids.append(a)
        else:
            october_ids.append(a)

    july_ids, october_ids = np.array(july_ids), np.array(october_ids)

    july_subsampler = torch.utils.data.SubsetRandomSampler(july_ids)
    october_subsampler = torch.utils.data.SubsetRandomSampler(october_ids)

    # Define data loaders for julying and testing data in this fold
    julyloader = torch.utils.data.DataLoader(TotalDataset, shuffle=False, batch_size=batch_s, sampler=july_subsampler)
    octoberloader = torch.utils.data.DataLoader(TotalDataset,shuffle=False, batch_size=batch_s, sampler=october_subsampler)

    print(f'Number of julying Data = {len(july_ids)}, and Number of october dataset = {len(october_ids)}')

## Show Video Sequence of a data Cube

In [ ]:
%matplotlib agg 
# To avoild display duplicate image beside video
def showSample(input_sample):
    numFrames = input_sample.shape[-1]
    fig = plt.figure()
    im = plt.imshow(input_sample[:,:,0], cmap='gray')

    def update(i):
        img = input_sample[:,:,i]
        im.set_data(img)
        return im

    ani = animation.FuncAnimation(fig, update, frames=numFrames, repeat=False)  
    display(HTML(ani.to_html5_video()))
    
print('Lenght of total Dataset = ',len(TotalDataset))

pos = 1700
aDataCube = TotalDataset[pos]['image'].cpu()
aDataCube = np.transpose(aDataCube, (1,2,0))
showSample(aDataCube)
%matplotlib inline

## Niave CNN Deep Learning

In [ ]:
# %load effnetv2.py
"""
Creates a EfficientNetV2 Model as defined in:
Mingxing Tan, Quoc V. Le. (2021). 
EfficientNetV2: Smaller Models and Faster Training
arXiv preprint arXiv:2104.00298.
import from https://github.com/d-li14/mobilenetv2.pytorch
"""

import torch
import torch.nn as nn
import math


classNumber = 1

__all__ = ['effnetv2_s', 'effnetv2_m', 'effnetv2_l', 'effnetv2_xl']


def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


# SiLU (Swish) activation function
if hasattr(nn, 'SiLU'):
    SiLU = nn.SiLU
else:
    # For compatibility with old PyTorch versions
    class SiLU(nn.Module):
        def forward(self, x):
            return x * torch.sigmoid(x)

 
class SELayer(nn.Module):
    def __init__(self, inp, oup, reduction=4):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
                nn.Linear(oup, _make_divisible(inp // reduction, 8)),
                SiLU(),
                nn.Linear(_make_divisible(inp // reduction, 8), oup),
                nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y


def conv_3x3_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        SiLU()
    )


class MBConv(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio, use_se):
        super(MBConv, self).__init__()
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.identity = stride == 1 and inp == oup
        if use_se:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                SELayer(inp, hidden_dim),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # fused
                nn.Conv2d(inp, hidden_dim, 3, stride, 1, bias=False),
                nn.BatchNorm2d(hidden_dim),
                SiLU(),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )


    def forward(self, x):
        if self.identity:
            return x + self.conv(x)
        else:
            return self.conv(x)


class EffNetV2(nn.Module):
    def __init__(self, cfgs, num_classes=classNumber, width_mult=1.):
        super(EffNetV2, self).__init__()
        self.cfgs = cfgs

        # building first layer
        input_channel = _make_divisible(24 * width_mult, 8)
        layers = [conv_3x3_bn(3, input_channel, 2)]
        # building inverted residual blocks
        block = MBConv
        for t, c, n, s, use_se in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 8)
            for i in range(n):
                layers.append(block(input_channel, output_channel, s if i == 0 else 1, t, use_se))
                input_channel = output_channel
        self.features = nn.Sequential(*layers)
        # building last several layers
        output_channel = _make_divisible(1792 * width_mult, 8) if width_mult > 1.0 else 1792
        self.conv = conv_1x1_bn(input_channel, output_channel)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(output_channel, num_classes)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.001)
                m.bias.data.zero_()


def effnetv2_s(**kwargs):
    """
    Constructs a EfficientNetV2-S model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  2, 1, 0],
        [4,  48,  4, 2, 0],
        [4,  64,  4, 2, 0],
        [4, 128,  6, 2, 1],
        [6, 160,  9, 1, 1],
        [6, 256, 15, 2, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_m(**kwargs):
    """
    Constructs a EfficientNetV2-M model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  24,  3, 1, 0],
        [4,  48,  5, 2, 0],
        [4,  80,  5, 2, 0],
        [4, 160,  7, 2, 1],
        [6, 176, 14, 1, 1],
        [6, 304, 18, 2, 1],
        [6, 512,  5, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_l(**kwargs):
    """
    Constructs a EfficientNetV2-L model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  7, 2, 0],
        [4,  96,  7, 2, 0],
        [4, 192, 10, 2, 1],
        [6, 224, 19, 1, 1],
        [6, 384, 25, 2, 1],
        [6, 640,  7, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)


def effnetv2_xl(**kwargs):
    """
    Constructs a EfficientNetV2-XL model
    """
    cfgs = [
        # t, c, n, s, SE
        [1,  32,  4, 1, 0],
        [4,  64,  8, 2, 0],
        [4,  96,  8, 2, 0],
        [4, 192, 16, 2, 1],
        [6, 256, 24, 1, 1],
        [6, 512, 32, 2, 1],
        [6, 640,  8, 1, 1],
    ]
    return EffNetV2(cfgs, **kwargs)

net = effnetv2_s().to(device)

## Deep Learning Model (Net -> net)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

def GradientDiffeciable(img):
    Ix = torch.diff(img, n=1, dim=-1)
    Ix = F.pad(input=Ix, pad=(0, 1, 0, 0), mode='replicate', value=0)

    Iy = torch.diff(img, n=1, dim=-2)
    Iy = F.pad(input=Iy, pad=(0, 0, 0, 1), mode='replicate', value=0)

    Grad_img = Ix**2 + Iy**2    # It's weird but they use not square root
    Grad_img[Grad_img < Grad_img.max()/100] = torch.nan
    return Grad_img

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(cubeChNum,GradientCh,kernelSize, padding='same')
    self.conv2 = nn.Conv2d(GradientCh,GradientCh,kernelSize, padding='same')
    self.conv3 = nn.Conv2d(GradientCh,GradientCh,kernelSize, padding='same')

  def forward(self, x):
    kvar = torch.var(x.squeeze(), dim=0)
    
    x = F.relu(self.conv1(x))    
    x = F.relu(self.conv2(x))    
    kGradStack = F.relu(self.conv3(x))
    kGradStack = torch.abs(kGradStack)
    
    kGradAvg = torch.nanmean(kGradStack, dim=0)
    kGradAvg[kGradAvg < kGradAvg.max()/100] = torch.nan
    kdiv = 0.00001/(torch.nanmean(kvar*kGradAvg))
    return kdiv#, kvar, kGradAvg, x

net = Net().to(device)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of Parameters = {count_parameters(net):,}')

## Load the Model if you wish

In [ ]:
LoadModel = False
if LoadModel:
    net.load_state_dict(torch.load('model/DiffGrad_O_AL0_abs_sin.pth'))
    plotTestResultSave(test_ids, f'{GraphName}, Epoch {epoch}')

## Optimizer

In [ ]:
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(net.parameters(), lr=lrate, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

## Test Accuracy Measurement/ Graph

In [ ]:
def saveResult(ori, pre, name):
    TestError = np.absolute(np.array(ori)-np.array(pre)).mean()
    TestError = str(np.round(TestError,18))
    #print(TestError)
    
    current_time = datetime.now().strftime("%D__%H-%M-%S")
    cTimeStr = str(current_time).replace('/','.')
    #print(cTimeStr)

    TestTable = {'Original': ori, 'Predicted': pre}
    testTableDf = pd.DataFrame(TestTable)
    testTableDf.to_csv(f"{GraphPath}{name}{cTimeStr}.csv")
    #print(f'Data saved to: {GraphPath}{name}{cTimeStr}.csv')
    
    plt.rcParams['figure.figsize'] = [20, 8]    
    plt.yscale('log',base=10), plt.grid(alpha=0.1)
    plt.plot(ori, label = "Original")
    plt.plot(pre, label = "Predicted")
    plt.title(f'{name} Error: {TestError}.png', size=20)
    plt.legend(), plt.grid()
    plt.savefig(f"{GraphPath}{name}{cTimeStr}.png")
    # print(f'Figure saved to: {GraphPath}/{name}{cTimeStr}.csv')
    plt.show()
    
def plotTestResultSave(DatasetIds, name):
    ind = []
    original = []
    predicted = []
    inc = 0

    for i in tqdm(DatasetIds, desc='Test '):
        inputs, labels = TotalDataset[i]['image'], TotalDataset[i]['features']
        
        
        # Multiplying with Distance factor of 2 when traingin on October and testing on July
        
        output = net(inputs.reshape(1,cubeChNum,256,256)).squeeze()*distanceFactor
        ind.append(i)
        original.append(Cn2deNorm(labels).cpu().detach().numpy().squeeze())
        predicted.append(Cn2deNorm(output).cpu().detach().numpy().squeeze())
        inc+=1
    saveResult(original, predicted, name)

## Training Interpolation

In [ ]:
# %%script false --no-raise-error
if DatasetName.value != 'Both':
    GraphName = f'DiffGrad_{DatasetName.value}'
    LossArray = []
    TestLossArray = []

    for epoch in range(epochCount):    
        net.train()
        running_loss = 0

        pbar = tqdm(trainloader, desc="Train")
        for i, data in (enumerate(pbar, 0)):
            optimizer.zero_grad()

            inputs, labels = data['image'], data['features']
            output = net(inputs).squeeze()
            loss = torch.mean(torch.absolute(labels-output)/torch.absolute(labels+output))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i%100 == 99:      
                pbar.set_postfix({'Loss100':  np.round(running_loss, 3)})
                running_loss = 0.0

        net.eval()
        plotTestResultSave(test_ids, f'GraphName, Epoch {epoch}')
    print('Finished Training')

## Training Extrapolaiton Model Transfer

In [ ]:
if DatasetName.value == 'Both':
    GraphName = f'DiffGrad_{DatasetName.value}'
    LossArray = []
    TestLossArray = []

    for epoch in range(epochCount):    
        net.train()
        running_loss = 0

        if Opposit==0:
            pbar = tqdm(julyloader, desc="Train")
        else:
            pbar = tqdm(octoberloader, desc="Train") 
        
        for i, data in (enumerate(pbar, 0)):
            optimizer.zero_grad()

            inputs, labels = data['image'], data['features']
            output = net(inputs).squeeze()
            loss = torch.mean(torch.absolute(labels-output))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i%100 == 99:      
                pbar.set_postfix({'Loss100':  np.round(running_loss, 3)})
                running_loss = 0.0

        net.eval()
        if Opposit==0:
            plotTestResultSave(october_ids, f'{GraphName}, Epoch {epoch}')
        else:
            plotTestResultSave(july_ids, f'{GraphName}, Epoch {epoch}')
    print('Finished Training')